In [180]:
from sympy import *
from IPython.display import display, Math

In [181]:
c = symbols('c', nonzero=True, real=True, infinite=False)
c1 = symbols('c_1', nonzero=True, real=True, infinite=False)
c2 = symbols('c_2', nonzero=True, real=True, infinite=False)
v = symbols('v', nonzero=True, real=True, infinite=False) #d was replaced with v for clarity. (v = normalize(pixel position - camera position))
sigma_t = symbols('sigma_t', nonzero=True, real=True, infinite=False)
o = symbols('o', real=True, infinite=False)
u = symbols('u', real=True)
t = symbols('t', real=True)
t2 = symbols('t_2', real=True)
D = symbols('D', real=True, infinite=False, positive=True)
a = symbols('a', real=True, infinite=False, positive=True)
fadeWidth = symbols('f_w', real=True, positive=True, nonzero=True)
b = (a+fadeWidth)
# b = symbols('b', real=True, infinite=False, positive=True)

In [182]:
def heightFactor(y):
    return exp(-y/c)

$ e^{- \int_{0}^{D}{\sigma_t(t) \mathrm{d}t}} $

Only height based

In [183]:
i = Integral(sigma_t*heightFactor(o+t*v), (t,0,D))
i

Integral(sigma_t*exp((-o - t*v)/c), (t, 0, D))

In [184]:
i.doit()

-c*sigma_t*exp((-D*v - o)/c)/v + c*sigma_t*exp(-o/c)/v

or in the case that v=0:

In [185]:
D*sigma_t*exp(-o/c)

D*sigma_t*exp(-o/c)

Adding a camera fade by scaling the "intensity" of the fog with a factor of $0$ to $1$ between distances $a$ and $b$ to the camera using a weighting factor $w(\mathrm{depth})$\
(which is of course not physical at all)

In [186]:
w = Function('w')

When integrating over $o+tv$, $t$ is already the distance from the camera since $v$ is normalized

In [187]:
generalIntegral = Integral(w(t)*sigma_t*heightFactor(o+t*v), (t,0,D))
generalIntegral

Integral(sigma_t*w(t)*exp((-o - t*v)/c), (t, 0, D))

which is the same as:

In [188]:
generalIntegral = Integral(w(t)*sigma_t*heightFactor(o+t*v), (t,0,a)) + Integral(w(t)*sigma_t*heightFactor(o+t*v), (t,a,b)) + Integral(w(t)*sigma_t*heightFactor(o+t*v), (t,b,D))
generalIntegral

Integral(sigma_t*w(t)*exp((-o - t*v)/c), (t, 0, a)) + Integral(sigma_t*w(t)*exp((-o - t*v)/c), (t, a, a + f_w)) + Integral(sigma_t*w(t)*exp((-o - t*v)/c), (t, a + f_w, D))

Where by definition of the fade, $w(t)$ is $0$ in the bounds of the first integral and $1$ in the bounds of the last integral.
This simplifies to 

In [189]:
generalIntegral = Integral(w(t)*sigma_t*heightFactor(o+t*v), (t,a,b)) + Integral(1*sigma_t*heightFactor(o+t*v), (t,b,D))
generalIntegral

Integral(sigma_t*exp((-o - t*v)/c), (t, a + f_w, D)) + Integral(sigma_t*w(t)*exp((-o - t*v)/c), (t, a, a + f_w))

Solving the last part:

In [190]:
endIntegral = Integral(sigma_t*heightFactor(o+t*v), (t,b,D))
endIntegral

Integral(sigma_t*exp((-o - t*v)/c), (t, a + f_w, D))

In [191]:
endIntegral.doit()

c*sigma_t*exp((-o - v*(a + f_w))/c)/v - c*sigma_t*exp((-D*v - o)/c)/v

Or in case $v=0$

In [192]:
simplify(endIntegral.subs(v,0).doit())

sigma_t*(D - a - f_w)*exp(-o/c)

Solving the middle integral depends on the choice of fade function

Simple linear ramp:

In [193]:
def linearRamp(x, start, end):
    return ((x - start) / (end - start))

In [194]:
middleIntegralLinear = Integral(linearRamp(t,a,b)*sigma_t*heightFactor(o+t*v), (t,a,b))
middleIntegralLinear

Integral(sigma_t*(-a + t)*exp((-o - t*v)/c)/f_w, (t, a, a + f_w))

In [195]:
middleIntegralLinear.doit()

c**2*sigma_t*exp((-a*v - o)/c)/(f_w*v**2) + (a*c*sigma_t*v - c**2*sigma_t - c*sigma_t*v*(a + f_w))*exp((-o - v*(a + f_w))/c)/(f_w*v**2)

TODO: find numerically stable version of this

Solving in case that $v=0$

In [197]:
middleIntegralLinear0 = middleIntegralLinear.subs(v,0)
middleIntegralLinear0

Integral(sigma_t*(-a + t)*exp(-o/c)/f_w, (t, a, a + f_w))

In [198]:
# middleIntegralLinear0.doit()
simplify(middleIntegralLinear0.doit())

f_w*sigma_t*exp(-o/c)/2

Another exception occurs when the pixel depth is lower than b, in that case the middle integral is bounded by the pixels depth instead

In [199]:
middleIntegralLinear2 = Integral(linearRamp(t,a,b)*sigma_t*heightFactor(o+t*v), (t,a,D))
middleIntegralLinear2

Integral(sigma_t*(-a + t)*exp((-o - t*v)/c)/f_w, (t, a, D))

In [200]:
middleIntegralLinear2.doit()
# middleIntegralLinear2.doit()

c**2*sigma_t*exp((-a*v - o)/c)/(f_w*v**2) + (-D*c*sigma_t*v + a*c*sigma_t*v - c**2*sigma_t)*exp((-D*v - o)/c)/(f_w*v**2)

In [201]:
middleIntegralLinear20 = middleIntegralLinear2.subs(v,0)
middleIntegralLinear20

Integral(sigma_t*(-a + t)*exp(-o/c)/f_w, (t, a, D))

In [202]:
# middleIntegralLinear20.doit()
simplify(middleIntegralLinear20.doit())

sigma_t*(D**2 - 2*D*a + a**2)*exp(-o/c)/(2*f_w)